# Parameters

Let's make sure that this is the only place where users have to change contents.

In [ ]:
# parameters

data_store = "gcs"

# Loading the data

In [ ]:
import intake

In [ ]:
catalog = intake.Catalog(f"intake-catalogs/medseaconnectivity_{data_store}.yaml")

In [ ]:
datasets = {k: catalog[k].to_dask() for k in catalog}
datasets

# Set up Dask cluster

In [ ]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client

In [ ]:
if data_store == "gcs":
    cluster = KubeCluster(n_workers=10)
    cluster.adapt(minimum=10, maximum=50)
    
client = Client(cluster)
display(client)

# Plot a heat map

We'll calculate a heat map data array and plot it.

In [ ]:
from xhistogram.xarray import histogram as xhist
import numpy as np
import hvplot.xarray
import holoviews as hv
hv.extension('bokeh')

In [ ]:
def calc_heat_map(dataset, lat_bins=None, lon_bins=None):
    # Calculate heat map
    heat_map = xhist(
        dataset.lat,
        dataset.lon,
        bins=[lat_bins, lon_bins],
        dim=["traj", "obs"]
    )
    
    # mask there no data
    heat_map = heat_map.where(heat_map > 0)
    
    heat_map = heat_map.rename("heat_map")
        
    return heat_map

In [ ]:
lat_bins = np.linspace(30, 50, 100)
lon_bins = np.linspace(0, 20, 100)

heat_map = calc_heat_map(
    datasets["medsea-trajectories-stokes"].isel(traj=slice(0, 200_000)),
    lat_bins=lat_bins,
    lon_bins=lon_bins
)

In [ ]:
heat_map.plot()